In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import pickle
import random
import json
import string

import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer

import nltk
from nltk.stem.lancaster import LancasterStemmer
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD

Using TensorFlow backend.


In [2]:
with open('kpi.json') as file:
    data = json.load(file)

In [3]:
stemmer = LancasterStemmer()

In [4]:
words = []
classes = []
documents = []
ignore_words = ["'s"]
ignore_words.extend(string.punctuation)

try:
    with open('data.pickle', 'rb') as file:
        words, classes, documents = pickle.load(file)
except:
    # loop through each sentence in our intents pattern
    for intent in data['intents']:
        for pattern in intent['patterns']:
            # tokenize each word in the sentence
            wrds = nltk.word_tokenize(pattern)
            words.extend(wrds) # add to word list
            documents.append((wrds, intent['tag'])) # add to documents in our corpus

        if intent['tag'] not in classes:
            classes.append(intent['tag']) # add to our class list

    # stem and lower each word and remove duplicates
    words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
    words = sorted(list(set(words)))

    classes = sorted(list(set(classes))) # sort classes

    print(f"{len(documents)} documents\n{len(classes)} classes\n{len(words)} unique stemmed words")


    with open('data.pickle', 'wb') as file:
        pickle.dump((words, classes, documents), file)
        
# create our training data
training = []
output_empty = [0 for _ in range(len(classes))]

for doc in documents: # bag of words
    bag = []
    s_words = [stemmer.stem(w.lower()) for w in doc[0] if w not in ignore_words]
    
    for w in words:
        bag.append(1) if w in s_words else bag.append(0)
        
    output_row = output_empty[:]
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
    
# shuffle our features and turn intp np.array
random.shuffle(training)
training = np.array(training)

# create train and test list.
train_x = list(training[:, 0])
train_y = list(training[:, 1])

model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

# Fit the model
model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
672/672 [==============================] - 3s 4ms/step - loss: 3.1608 - accuracy: 0.1577
Epoch 2/200
672/672 [==============================] - 1s 1ms/step - loss: 2.4131 - accuracy: 0.3095
Epoch 3/200
672/672 [==============================] - 1s 1ms/step - loss: 1.8139 - accuracy: 0.4301
Epoch 4/200
672/672 [==============================] - 1s 1ms/step - loss: 1.3226 - accuracy: 0.5997
Epoch 5/200
672/672 [==============================] - 1s 1ms/step - loss: 1.0194 - accuracy: 0.7113
Epoch 6/200
672/672 [==============================] - 1s 1ms/step - loss: 0.8622 - accuracy: 0.7500
Epoch 7/200
672/672 [==============================] - 1s 1ms/step - loss: 0.7277 - accuracy: 0.7946
Epoch 8/200
672/672 [==============================] - 1s 1ms/step - loss: 0.6235 - accuracy: 0.8214
Epoch 9/200
672/672 [==============================] - 1s 972us/step - loss: 0.5395 - accuracy: 0.8438
Epoch 10/200
672/672 [==============================] - 1s 944us/step - loss: 0.5188 - ac

In [ ]:
def clean_up_sentence(sentence):
    s_words = nltk.word_tokenize(sentence)
    s_words = [stemmer.stem(w.lower()) for w in s_words if w not in ignore_words]
    
    return s_words

def bow(sentence, words, show_details=True):
    s_words = clean_up_sentence(sentence)
    
    bag = [0 for _ in range(len(words))]
    
    for s in s_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                
                if show_details:
                    print(f"found in bag: {s}")
                    
    return np.array(bag)

def classify_local(sentence):
    ERROR_THRESHOLD = 0.25
    
    input_data = pd.DataFrame([bow(sentence, words, show_details=False)], dtype=float, index=['input'])
    results = model.predict([input_data])[0]
    
    #filter out prediction below a threshold, and provide intent index
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    
    return_list = []
    for r in results:
        return_list.append((classes[r[0]], str(r[1])))
    
    return return_list
    
while True:
    inp = input("You: ")
    
    if inp == 'quit' or inp == 'stop':
        break
        
    results = classify_local(inp)[0]
    
    for intent in data['intents']:
        if intent['tag'] == results[0]:
            responses = intent['responses']
    
    print(random.choice(responses))
    

You:  whats my expensive of yesterday?


yesterday's profit was


You:  what's my expenses for yesterday?


yesterday's cost was
